# Examples of using the model inspection class and the profiling functions

The inspection class can gather model parameters, gradients, activations and activation gradients.

The profiling module provide information about cpu and GPU timing, memory usage and flops operations.

In [1]:
import torch
from torchtrainer.models.resunet import ResUNetV2
from torchtrainer.inspector import Inspector
from torchtrainer.profiling import benchmark_model

# Dummy data. In a real application this should be a batch of the dataset.
batch = torch.rand(8, 1, 224, 224)
labels = torch.randint(0, 1000, (8,))

model = ResUNetV2((3, 3, 3), (1, 1, 1), (16, 32, 64))
model.eval();

### Model inspection

In [2]:
# Check intermediate activations of the model, as well as gradients and parameters
insp = Inspector(model)
# For activations, we need to explicitly start tracking to set up the forward hooks.
insp.start_tracking_activations()

# Apply model to batch
res = model(batch)
# Dummy calculation of loss and gradients, just as an example.
loss = res.sum().backward()

# Remove the forward hooks
insp.stop_tracking_activations()

# Activations
acts = insp.get_activations()
# Parameters
params = insp.get_params()
# Gradients
grads = insp.get_grads()

names, values = zip(*acts)
print(names[:10])    # Names of the first 10 layers
print(values[0].shape)  # Size of the activations of the first layer

('resunetv2.stage_input.0', 'resunetv2.stage_input.1', 'resunetv2.encoder.stage_0.0.conv1', 'resunetv2.encoder.stage_0.0.bn1', 'resunetv2.encoder.stage_0.0.conv2', 'resunetv2.encoder.stage_0.0.bn2', 'resunetv2.encoder.stage_0.0.residual_adj.0', 'resunetv2.encoder.stage_0.0.residual_adj.1', 'resunetv2.encoder.stage_0.0', 'resunetv2.encoder.stage_0.1.conv1')
torch.Size([8, 16, 224, 224])


Tracking activations involves copying all the data from the GPU to the CPU (to preserve GPU memory), which is expensive. We can provide and aggregation function that will be applied to the data before copying

In [3]:
def agg_func(data, module_name, data_type):
    return torch.tensor([data.min(), data.max()])

insp = Inspector(model, agg_func=agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('resunetv2.stage_input.0', tensor([-1.0450,  0.7769])), ('resunetv2.stage_input.1', tensor([-1.0450,  0.7769])), ('resunetv2.encoder.stage_0.0.conv1', tensor([-0.7046,  0.8440])), ('resunetv2.encoder.stage_0.0.bn1', tensor([-0.7046,  0.8440])), ('resunetv2.encoder.stage_0.0.conv2', tensor([-0.8168,  0.7232])), ('resunetv2.encoder.stage_0.0.bn2', tensor([-0.8168,  0.7232])), ('resunetv2.encoder.stage_0.0.residual_adj.0', tensor([-0.9841,  0.7573])), ('resunetv2.encoder.stage_0.0.residual_adj.1', tensor([-0.9841,  0.7573])), ('resunetv2.encoder.stage_0.0', tensor([0.0000, 1.2535])), ('resunetv2.encoder.stage_0.1.conv1', tensor([-1.1476,  1.4517])), ('resunetv2.encoder.stage_0.1.bn1', tensor([-1.1476,  1.4517])), ('resunetv2.encoder.stage_0.1.conv2', tensor([-1.8410,  1.1853])), ('resunetv2.encoder.stage_0.1.bn2', tensor([-1.8410,  1.1853])), ('resunetv2.encoder.stage_0.1', tensor([0.0000, 1.4357])), ('resunetv2.encoder.stage_0.2.conv1', tensor([-1.3182,  1.6929])), ('resunetv2.encoder.

We can also only track individual modules

In [5]:
insp = Inspector(model, [model.stage_input,model.encoder.stage_2[0]], agg_func)
insp.start_tracking_activations()
res = model(batch)
insp.stop_tracking_activations()

acts = insp.get_activations()
print(acts)

[('resunetv2.stage_input', tensor([0.0000, 0.7769])), ('resunetv2.encoder.stage_2.0', tensor([0.0000, 4.2164]))]


It is also possible to track activation gradients, but it is important to note that they do not work for a layer if the previous layer has an inplace operation.

In [6]:
insp = Inspector(model, [model.stage_input])
insp.start_tracking_act_grads()
res = model(batch)
insp.stop_tracking_act_grads()
loss = res.sum().backward()

act_grads = insp.get_act_grads()

### Model profiling

In [7]:
tensor_shape = (8, 1, 224, 224)
# Benchmark the model for training
stats_train = benchmark_model(model, tensor_shape, no_grad=False, call_backward=True, use_float16=True, return_model_info=True)
# Benchmark for inference
stats_val = benchmark_model(model, tensor_shape, no_grad=True, call_backward=False, use_float16=True, return_model_info=False)
# The units for each metric are also included in the dictionary
print(stats_train)
print(stats_val)

Unsupported operator aten::batch_norm encountered 33 time(s)
Unsupported operator aten::add_ encountered 12 time(s)
Unsupported operator aten::upsample_nearest2d encountered 3 time(s)
Unsupported operator aten::add_ encountered 12 time(s)


{'params': 0.320336, 'activations': 0.053989376, 'flops': 9.023250432, 'memory': 0.3600749969482422, 'time_cpu': 0.029000043869018555, 'time_gpu': 0.02601759910583496, 'info': ['params: M', 'activations: G', 'flops: G', 'memory: GiB', 'time_cpu: s', 'time_gpu: s']}
{'params': 0.0, 'activations': 0.0, 'flops': 0.0, 'memory': 0.1539907455444336, 'time_cpu': 0.007998943328857422, 'time_gpu': 0.007321887969970703, 'info': ['params: M', 'activations: G', 'flops: G', 'memory: GiB', 'time_cpu: s', 'time_gpu: s']}
